In [1]:
from pathlib import Path
import subprocess
import os
from glob import glob
import numpy as np
import pandas as pd
from datetime import datetime
import joblib
import warnings
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm
from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.datasets import load_wine
import pickle
import seaborn as sns

In [2]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("nllb-200-3.3B/tokenize")

model = torch.load("Qwen/chinda-qwen3-4b.pt", weights_only = False)
model.to(device)
model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560, padding_idx=151643)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
        (post_atten

In [3]:
import torch
print(torch.__version__)           # Should show 2.2.2+cu118
print(torch.cuda.is_available())   # Should return True

2.6.0+cu124
True


In [4]:
data_train = pd.read_json(path_or_buf="Data_MT/mt_train.jsonl", lines=True)
data_val = pd.read_json(path_or_buf="Data_MT/mt_dev.jsonl", lines=True)
data_test = pd.read_json(path_or_buf="Data_MT/mt_test.jsonl", lines=True)

In [5]:
data_train = data_train.drop('context', axis=1)
data_val = data_val.drop('context', axis=1)

In [6]:
data_train

,source,translation
0,我想咨询件事 我对别人有点敌意,ฉันอยากจะถามอะไรหน่อย ฉันรู้สึกไม่เป็นมิตรกับค...
1,我考虑细菌性或滴虫性阴道炎的可能性,ฉันกำลังคิดว่าน่าจะเกิดภาวะช่องคลอดอักเสบจากเช...
2,你好！腰痛发现有多长时间了？是腰中间疼痛还是两侧肌肉疼痛？,สวัสดี คุณมีอาการปวดหลังส่วนล่างมานานแค่ไหนแล้...
3,是不是平时勤刷牙，多刷干净点就可以减少牙结石的沉积呢？,ถ้าแปรงฟันบ่อยๆ และทำความสะอาดบ่อยๆ ก็จะสามารถ...
4,周围环境的物理化学因素也可以引起身体过敏,ปัจจัยทางกายภาพและเคมีในสภาพแวดล้อมโดยรอบ ก็สา...
...,...,...
18595,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้ฉันค่อนข้างยุ่...
18596,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้หมอค่อนข้างยุ่...
18597,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้หมอค่อนข้างยุ่...
18598,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้ฉันค่อนข้างยุ่...


In [7]:
from datasets import Dataset, DatasetDict
from transformers import (
    NllbTokenizer, 
    M2M100ForConditionalGeneration,
    Trainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import logging
from IPython.display import display, HTML
import matplotlib.pyplot as plt

class Config:
    SRC_LANG = "zho_Hans"  # Simplified Chinese
    TGT_LANG = "tha_Thai"  # Thai
    
    # Training parameters
    OUTPUT_DIR = "./1.3B-Out-cosine"
    NUM_EPOCHS = 30
    BATCH_SIZE = 2  # Adjust based on your GPU memory
    LEARNING_RATE = 1e-5
    MAX_LENGTH = 1140

config = Config()

In [8]:
data_val['translation']

0       ตาพร่ามัวและหมดสติเนื่องจากสมองบาดเจ็บ (ชายอาย...
1       เดือนที่แล้ว ประจำเดือนครั้งแรกมาตอนต้นเดือน ค...
2                  บางคนแพ้ ทำแล้วไม่สบาย อึดอัดและปวดบวม
3       ความเครียดเยอะ เบื่ออาหารและน้ำ จะทำให้ผอมลงแล...
4              ความรู้สึกตอนนี้คือเหมือนมีอาการประสาทหลอน
                              ...                        
2995    สวัสดี หินปูนเป็นรอยโรคที่หลงเหลืออยู่หลังจากก...
2996            พี่สาวของฉันบอกให้กิน เธอเป็นสูตินรีแพทย์
2997          วิธีการรักษาโรคประสาทอ่อน? (ชาย อายุ 31 ปี)
2998    หลังการสัมผัสเชื้อมา 3 เดือนสามารถยืนยันได้อย่...
2999    ข้อใดข้อหนึ่งก็สามารถตัดความเสี่ยงของการติดเชื...
Name: translation, Length: 3000, dtype: object

In [9]:
x_train, x_test, y_train, y_test = train_test_split(
    data_val['source'], data_val['translation'], 
    test_size=0.05,  # 20% for test set
    random_state=42,  # for reproducibility
    shuffle=True,
)

In [10]:
result = pd.concat([data_train, pd.DataFrame({"source": x_train, "translation": y_train})], axis=0)
# result = pd.concat([data_train, data_val, axis=0)


In [11]:
datasets = DatasetDict({
    'train': Dataset.from_dict({'source': result['source'], 'translation': result['translation']}),
    'validation': Dataset.from_dict({'source': x_test, 'translation': y_test}),
})

In [12]:
datasets

DatasetDict({
    train: Dataset({
        features: ['source', 'translation'],
        num_rows: 21450
    })
    validation: Dataset({
        features: ['source', 'translation'],
        num_rows: 150
    })
})

In [13]:
tokenizer.src_lang = config.SRC_LANG
tokenizer.tgt_lang = config.TGT_LANG

In [14]:
# Tokenize with context (Option 2 - Batched)
def tokenize_function(examples):
    inputs = tokenizer(
        examples["source"],
        truncation=True,
        max_length=1140,
        return_tensors="pt",
        padding="max_length", 
    )
    labels = tokenizer(
        examples["translation"],
        truncation=True,
        max_length=1140,
        return_tensors="pt",
        padding="max_length"
    )
    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply to dataset
tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=datasets["train"].column_names,
)

Map:   0%|          | 0/21450 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [15]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21450
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
})

In [16]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="max_length"
)

In [17]:
training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,
    num_train_epochs=config.NUM_EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=8,
    warmup_steps=1000,
    learning_rate=config.LEARNING_RATE,
    weight_decay=0.01,
    logging_dir=f"{config.OUTPUT_DIR}/logs",
    logging_steps=1000,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=torch.cuda.is_available(),
    report_to="tensorboard",
    remove_unused_columns=False,                             # Saves memory; set False if using custom dataset fields
    dataloader_pin_memory=True,                             # Helpful on some systems
    gradient_checkpointing=True,
    dataloader_num_workers=2,                               # Speeds up data loading; adjust as needed
    lr_scheduler_type="cosine",
    optim="adamw_torch",
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

/tmp/ipykernel_53199/2915407132.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
def print_trainable_parameters(model):
    trainable = 0
    total = 0
    for param in model.parameters():
        total += param.numel()
        if param.requires_grad:
            trainable += param.numel()
    print(f"Trainable parameters: {trainable:,}")
    print(f"Total parameters:     {total:,}")
    print(f"Trainable %:          {100 * trainable / total:.2f}%")

print_trainable_parameters(model)

Trainable parameters: 1,370,638,336
Total parameters:     1,370,638,336
Trainable %:          100.00%


In [20]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,8.744800,2.073067
2,1.612400,0.043583
3,0.037400,0.037017
4,0.032800,0.034402
5,0.030300,0.033315
6,0.026800,0.032405
7,0.024900,0.031716
8,0.023600,0.031453
9,0.021500,0.031519
10,0.020200,0.031520


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/home/tb1017/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the follo

TrainOutput(global_step=13410, training_loss=0.7965295803004641, metrics={'train_runtime': 36697.2942, 'train_samples_per_second': 17.535, 'train_steps_per_second': 1.096, 'total_flos': 1.62605122707456e+18, 'train_loss': 0.7965295803004641, 'epoch': 10.0})

In [21]:
modelx = trainer.model.to("cuda")

In [22]:
modelx

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-23): 24 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=1024, bias=True)
   

In [24]:
import json
def translate_text(chinese_text):
    # Tokenize input
    inputs = tokenizer(
        chinese_text, 
        truncation=True, 
        max_length=config.MAX_LENGTH,
        padding="max_length",
        return_tensors="pt"              # Ensure PyTorch tensors
    ).to("cuda")
    
    # Move to GPU if available
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        modelx.cuda()
        
    # Generate translation
    with torch.no_grad():
        generated_tokens = modelx.generate(
            **inputs,
            forced_bos_token_id=256175,
            num_beams=10,  # ← This parameter
            temperature=0.7,
            early_stopping=True
        )
    
    # Decode the translation
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translation

In [25]:
translate_text(x_test[0])

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'อาการบาดเจ็บที่สมองทําให้มองเห็นไม่ชัดและไม่สะอาด (ชาย อายุ 40 ปี)'

In [26]:
import tqdm
predictions = []
for text in tqdm.tqdm(x_test):
    predictions.append(translate_text(text))

predictions

100%|██████████| 150/150 [02:29<00:00,  1.00it/s]


['บางครั้งในตอนเช้าจะมีอาการน้ํามูกเหลืองไม่มาก ส่วนเวลาอื่นไม่มีอาการเหลือง',
 'ถ้าอย่างนั้นก็เป็นโรคเบาหวานจริงๆ และอาจเป็นมาก่อนตั้งครรภ์',
 'การตรวจปัสสาวะครั้งก่อนของฉันมีโปรตีนในปัสสาวะ 1 บวก ซึ่งเป็นมาครึ่งเดือนแล้ว',
 'งั้นก็ไม่ใช่ปัญหาใหญ่เลยใช่ไหม?',
 'ไม่มีอะไรที่มีประสิทธิภาพเป็นพิเศษ ทําการติดตามและตรวจซ้ํา',
 'แล้วตอนนี้จะรักษาอย่างไรดี และจะมีชีวิตอยู่ได้นานแค่ไหน?',
 'อาการแบบคุณน่าจะเป็นความผิดปกติของระบบประสาท',
 'คุณหมอคะ ฉันคลอดตามธรรมชาติ ตอนคลอดรู้สึกอยากกลับมาเป็นซ้ําเล็กน้อย แต่ไม่มีอาการ จะไม่ส่งผลกระทบต่อลูกใช่ไหมคะ',
 'ด้วยความยินดี หากเป็นประโยชน์กับคุณ โปรดให้คะแนนดีๆ ขอบคุณ!',
 'จากคําอธิบายของอัลตราซาวด์ น่าจะเป็นไขมันในตับ',
 'นอกจากนี้ อาการหูหนวกกะทันหันค่อนข้างยากที่จะรักษา ไม่ต้องกดเข็มให้กับเขา สามารถทานยานี้ได้ และต้องได้รับการรักษาด้วยการฉีดยา',
 'การได้ยินของฉันไม่ชัดเจน ไม่ใช่ว่าฉันไม่ได้ได้ยิน',
 'ผมร่วงจากต่อมไขมันคืออะไร ใช้ครีมบํารุงศีรษะชนิดใด (เพศหญิง อายุ 20 ปี)',
 'ก่อนหน้านี้ หมอบอกว่าปากมดลูกสั้นมากประมาณครึ่งเซนติเมตร แล้วก็โยกลง',
 '

In [27]:
ref = []
ref = y_test
ref

1801    ตอนเช้าจะมีน้ำมูกเขียวแต่ไม่มาก เวลาอื่นไม่มีน...
1190    แบบนี้แสดงว่าเป็นโรคเบาหวานจริง ๆ และอาจเป็นมา...
1817    ช่วงก่อนหน้านี้ ผลตรวจปัสสาวะของฉันพบว่ามีโปรต...
251               ถ้าอย่างนั้นก็ไม่มีปัญหาใหญ่อะไรใช่ไหม?
2505    ไม่มีอะไรมีสรรพคุณพิเศษ ทำได้แต่ติดตามผลการตรวจไป
                              ...                        
450           สวัสดี ทารกคลอดตอนครบกำหนดหรือคลอดก่อนกำหนด
2164    คุณอายุยังน้อยก็เป็นโรคหลอดเลือดสมองตีบแล้วหรื...
2372    ถ้าให้ดี ขอดูภาพลิ้นหน่อย จะได้ช่วยวินิจฉัยได้...
2012    ไปตรวจตอนที่ปวดแล้ว กระดูกสันหลังส่วนเอวไม่มีป...
270     เลือดเป็นสีแดงสดไหม? เป็นหยดๆ หรือพุ่งออกมา หร...
Name: translation, Length: 150, dtype: object

In [28]:
syn = {
    '้ํา' : '้ำ',
    '่ํา' : '่ำ',
    '๊ํา' : '๊ำ',
    '๋ํา' : '๋ำ',
    'ํา' : 'ำ'
}
mem = []
for data in predictions:
    s = data
    for k,v in syn.items(): s = s.replace(k,v)
    mem.append(s)
predictions = mem

In [29]:
from pythainlp.tokenize import word_tokenize
import sacrebleu

# Tokenize predictions and references
tokenized_preds = [" ".join(word_tokenize(pred)) for pred in predictions]
tokenized_refs = [" ".join(word_tokenize(r)) for r in ref]

# Compute BLEU with no tokenization (since we already tokenized)
score = sacrebleu.corpus_bleu(tokenized_preds, [tokenized_refs], tokenize="none")
print(f"BLEU score: {score.score:.2f}")
#29.02

BLEU score: 28.55


In [30]:
with open("summit_MT/nllb-200-1.3B(fine-tune-cosine)(val).txt", "w", encoding="utf-8") as f:
    for item in predictions:
        f.write(f"{item}\n")

In [ ]:
answer = []
for text in tqdm.tqdm(data_test['source']):
    answer.append(translate_text(text))

 34%|███▍      | 687/2000 [10:59<19:33,  1.12it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
translate_text(data_test['source'][0])

In [ ]:
mem = []
for data in answer:
    s = data
    for k,v in syn.items(): s = s.replace(k,v)
    mem.append(s)
answer = mem

In [ ]:
with open("summit_MT/nllb-200-1.3B(fine-tune-cosine).txt", "w", encoding="utf-8") as f:
    for item in answer:
        f.write(f"{item}\n")